In [1]:
# Import dependencies
import requests
import pandas as pd
import numpy as np

In [2]:
# Base url for the api call
url = "https://clinicaltrials.gov/api/query/study_fields"

In [3]:
# Headers for json call
headers = {'Content-Type': 'application/json', 
           'Accept': 'application/json'}

In [4]:
# Extract data and create DataFrame
def create_df(x):

    # Create a list to hold the dictionaries from the json response
    sourced_data = []

    # Set a counter to increase the ranks with each iteration
    # Table search criteria limited to 'breast cancer' and from 01/01/2018
    counter = 0

    while True:
        # Define the parameters of the url
        params = {'expr': 'breast cancer AND AREA[StartDate]RANGE[01/01/2018,MAX]', 
                'fields': ','.join(x), 
                'min_rnk': 1 + 1000 * counter,  
                'max_rnk': 1000 + 1000 * counter, 
                'fmt': 'json'}

        # Make the call 
        response = requests.get(url, 
                                headers = headers, 
                                params = params)

        

        # Increment the counter
        counter += 1
        
        # Check to see if more data was returned for the current response
        if response.json()['StudyFieldsResponse']['NStudiesReturned'] == 0:
            # If there was no new data returned, break out of the while loop
            break

        # Add the data from the api call to the sourced_data list
        sourced_data += response.json()['StudyFieldsResponse']['StudyFields']
    return sourced_data

In [5]:
# Define function to clean DataFrame removing unnecessary characters for analysis

def clean_data(df, fields):
    # Change datatype of the values, preparing for reg expression
    df[fields] = df[fields].astype(str)

    # Remove any unnecessary characters and turn blank values with NaN (null)
    for i in fields:
        df[i] = df[i].str.replace("^\[.|.\]$|'","")
        df[i] = df[i].replace('',np.nan)
        

In [6]:
# Define function to clean DataFrame removing unnecessary characters for analysis

def clean_data_years(df, fields):
    # Change datatype of the values, preparing for reg expression
    df[fields] = df[fields].astype(str)

    # Remove any unnecessary characters and turn blank values with NaN (null)
    for i in fields:
        df[i] = df[i].str.replace('Years', '')
        df[i] = df[i].str.replace('Year', '')

In [7]:
# List of fields for table A
table_a_cols = ['OrgStudyId',
               'BriefTitle',
               'StartDate',
               'CompletionDate',
               'OverallStatus',
               'StudyType']

In [7]:
# Call the function the create DataFrame for table A
registration_df = pd.DataFrame(create_df(table_a_cols))
registration_df.set_index('Rank', inplace=True)

In [9]:
# Clean DataFrame for table A
clean_data(registration_df, table_a_cols)

/Users/annekahjones/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":


In [10]:
# Renaming the columns for table A and convertion to datetime and removing duplicates
registration_df.columns = ['ID', 'Title', 'Start Date', 'Completion Date', 'Status', 'Study Type']
registration_df.index.names = ['Index']
registration_df['Start Date'] = pd.to_datetime(registration_df['Start Date'])
registration_df['Completion Date'] = pd.to_datetime(registration_df['Completion Date'])
registration_df.drop_duplicates(subset=['ID'], keep='first')
registration_df

,ID,Title,Start Date,Completion Date,Status,Study Type
Index,,,,,,
1,BTX-BCI-016-PRT,Breast Cancer Index (BCI) Registry,2021-04-14,2028-12-01,Recruiting,Observational
2,2018-TJ-BCD,Diagnosis Value of SEMA4C in Breast Cancer,2023-09-01,2024-09-01,Not yet recruiting,Observational
3,Breast cancer,Role of Sorcin and Annexin A3 in Breast Cancer...,2019-01-20,2019-09-30,Unknown status,Observational
4,BC-BOMET,Evaluation of Prognostic Factors: From Breast ...,2020-01-13,2024-11-12,Recruiting,Observational
5,241391,A Study to Identify Breast Cancer (IDBC),2019-01-24,2022-12-31,Unknown status,Observational
...,...,...,...,...,...,...
5009,NCI-2022-02915,Testing the Combination of the Anti-cancer Dru...,"August 4, 2022","August 4, 2025",Recruiting,Interventional
5010,4-2017-0840,Efficacy and Safety of Modified Nab-Paclitaxel...,"April 1, 2018",June 2020,Unknown status,Interventional
5011,21953,A Study Called ARAMON to Learn to What Extent ...,"December 19, 2022","October 20, 2025",Recruiting,Interventional


In [10]:
# List of fields for table B
table_b_cols = ['OrgStudyId',
               'Gender',
               'MinimumAge',
               'HealthyVolunteers']

In [11]:
# Call the function the create DataFrame for table B
participant_df = pd.DataFrame(create_df(table_b_cols))
participant_df.set_index('Rank', inplace=True)

In [12]:
# Clean DataFrame for table B
clean_data(participant_df, table_b_cols)
clean_data_years(participant_df, table_b_cols)

/Users/annekahjones/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":


In [13]:
# Renaming the columns for table B and getting rid of rows
participant_df.columns = ['ID', 'Gender', 'Minimum Age', 'Healthy Volunteers']
participant_df.index.names = ['Index']
participant_df = participant_df[participant_df['Minimum Age'].str.contains('Months') == False]
participant_df.drop_duplicates(subset=['ID'], keep='first')
participant_df

,ID,Gender,Minimum Age,Healthy Volunteers
Index,,,,
1,BTX-BCI-016-PRT,Female,18,No
2,2018-TJ-BCD,Female,18,No
3,Breast cancer,Female,20,Accepts Healthy Volunteers
4,BC-BOMET,Female,18,No
5,241391,Female,30,Accepts Healthy Volunteers
...,...,...,...,...
5009,NCI-2022-02915,All,18,No
5010,4-2017-0840,All,19,No
5011,21953,Male,18,No


In [14]:
# List of fields for table C
table_c_cols = ['OrgStudyId',
               'IsFDARegulatedDrug',
               'IsFDARegulatedDevice',
               'ResponsiblePartyType']

In [15]:
# Call the function the create DataFrame for table C
study_details_df = pd.DataFrame(create_df(table_c_cols))
study_details_df.set_index('Rank', inplace=True)

In [16]:
# Clean DataFrame for table C
clean_data(study_details_df, table_c_cols)

/Users/annekahjones/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":


In [17]:
# Renaming the columns for table C and removing duplicates
study_details_df.columns = ['ID', 'FDA Regulated Drug?', 'FDA Regulated Device?', 'Responsible Party']
study_details_df.index.names = ['Index']
study_details_df.drop_duplicates(subset=['ID'], keep='first')
study_details_df

,ID,FDA Regulated Drug?,FDA Regulated Device?,Responsible Party
Index,,,,
1,BTX-BCI-016-PRT,No,No,Sponsor
2,2018-TJ-BCD,No,No,Principal Investigator
3,Breast cancer,No,No,Principal Investigator
4,BC-BOMET,No,No,Sponsor
5,241391,No,No,Sponsor
...,...,...,...,...
5009,NCI-2022-02915,Yes,No,Sponsor
5010,4-2017-0840,No,No,Sponsor
5011,21953,Yes,No,Sponsor


In [18]:
# List of fields for table d
table_d_cols = ['OrgStudyId',
          'StudyType',
          'ArmGroupType',
          'InterventionType',
          'DesignInterventionModel',
          'DesignObservationalModel',
          'TargetDuration',
          'SamplingMethod',
          'Phase']

# Call function to create DataFrame for table D
study_method_df = pd.DataFrame(create_df(table_d_cols))
study_method_df.set_index('Rank',inplace=True)

In [19]:
# Clean DataFrame for table c
clean_data(study_method_df, table_d_cols)

/Users/annekahjones/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":


In [20]:
# Renaming the columns for table D and removing duplicates
study_method_df.columns = ['ID', 'Study Type', 'Arm Group Type', 'Intervention Type', 'Interventional Study Model', 'Observational Study Model', 'Target Duration', 'Sampling Method', 'Phase']
study_method_df.index.names = ['Index']
study_method_df.drop_duplicates(subset=['ID'], keep='first')
study_method_df

,ID,Study Type,Arm Group Type,Intervention Type,Interventional Study Model,Observational Study Model,Target Duration,Sampling Method,Phase
Index,,,,,,,,,
1,BTX-BCI-016-PRT,Observational,NaN,Diagnostic Test,NaN,Case-Only,5 Years,Non-Probability Sample,NaN
2,2018-TJ-BCD,Observational,NaN,"Diagnostic Test, Diagnostic Test",NaN,Case-Control,NaN,Non-Probability Sample,NaN
3,Breast cancer,Observational,NaN,"Genetic, Other",NaN,Case-Control,NaN,Probability Sample,NaN
4,BC-BOMET,Observational,NaN,"Other, Other",NaN,Case-Control,NaN,Non-Probability Sample,NaN
5,241391,Observational,NaN,Diagnostic Test,NaN,Cohort,1 Year,Non-Probability Sample,NaN
...,...,...,...,...,...,...,...,...,...
4999,32900654326,Interventional,"Placebo Comparator, Active Comparator, Active ...","Procedure, Drug",Parallel Assignment,NaN,NaN,NaN,Not Applicable
5000,ReDA 13176,Interventional,"Active Comparator, Experimental","Radiation, Drug, Drug, Drug, Drug, Other, Radi...",Parallel Assignment,NaN,NaN,NaN,Phase 3
5001,64121317.4.1001.5330,Interventional,Experimental,Drug,Single Group Assignment,NaN,NaN,NaN,Phase 2


In [21]:
# List of fields for table e and export dataset into DataFrame
table_e_cols = ['OrgStudyId',
          'EnrollmentCount',
          'PrimaryOutcomeMeasure',]
free_text_df = pd.DataFrame(create_df(table_e_cols))
free_text_df.set_index('Rank', inplace=True)


In [22]:
clean_data(free_text_df, table_e_cols)

/Users/annekahjones/opt/anaconda3/envs/PythonData/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":


In [23]:
# Renaming the columns for table E and removing duplicates
free_text_df.columns = ['ID', 'Enrollment Count', 'Primary Outcome measure']
free_text_df.index.names = ['Index']
free_text_df.drop_duplicates(subset=['ID'], keep='first')
free_text_df

ID Why Study Stopped Enrollment Count  \
Rank                                                       
1     BTX-BCI-016-PRT               NaN             3000   
2         2018-TJ-BCD               NaN             2300   
3       Breast cancer               NaN               80   
4            BC-BOMET               NaN               30   
5              241391               NaN              600   
...               ...               ...              ...   
5009   NCI-2022-02915               NaN               88   
5010      4-2017-0840               NaN               52   
5011            21953               NaN               65   
5012            J1940               NaN               60   
5013         2018PS02               NaN               19   

                                Primary Outcome Measure Reason Not Completed  
Rank                                                                          
1     To determine BCI test performance by evaluatin...                  NaN  
2     Diagnostic potential of SEMA4C as a biomarker ...                  NaN  
3         Role of SORCIN in patients with breast cancer                  NaN  
4                                      SENP1 expression                  NaN  
5     Performance of the Syantra DX Breast Cancer te...                  NaN  
...                                                 ...                  ...  
5009                      Objective response rate (ORR)                  NaN  
5010      Objective response rate, Disease control rate                  NaN  
5011  Lead-in phase: Change in serum testosterone, R...                  NaN  
5012  Feasibility of study as assessed by completion...                  NaN  
5013  Initially targeted drug therapy risks in patie...                  NaN  

[5013 rows x 3 columns]

In [24]:
# Export DataFrames as csv files 
study_method_df.to_csv('Tables/study_method_df.csv',index=True)
free_text_df.to_csv('Tables/free_text_df.csv', index=True)
registration_df.to_csv('Tables/registration_df.csv', index=True)
participant_df.to_csv('Tables/participant_df.csv', index=True)
study_details_df.to_csv('Tables/study_details_df.csv', index=True)